# Step 1: Sign up for a Twitter developer account.

In order to pull your data, you'll need your keys from a Twitter developer account. 

You'll need the following:
API/Consumer Key - developer account specific
API/Consumer Secret - developer account specific
Access Key - Twitter account specific
Access Secret - Twitter account specific

OR 

Bearer token

In [46]:
import os
import tweepy
import pandas as pd

In [ ]:
USER_FIELDS="id,name,username,description,url,created_at,public_metrics,profile_image_url,pinned_tweet_id,location"
TWITTER_API_KEY=os.environ.get('TWITTER_API_KEY')
TWITTER_API_SECRET=os.environ.get('TWITTER_API_SECRET')
TWITTER_ACCESS_TOKEN=os.environ.get('TWITTER_ACCESS_TOKEN')
TWITTER_ACCESS_SECRET=os.environ.get('TWITTER_ACCESS_SECRET')
TWITTER_BEARER_TOKEN=os.environ.get('TWITTER_BEARER_TOKEN')
TWITTER_USERNAME = "YOUR_TWITTER_USERNAME"


In [38]:


client = tweepy.Client(TWITTER_BEARER_TOKEN)
# Alternate approach
# client = tweepy.Client(consumer_key=TWITTER_API_KEY, consumer_secret=TWITTER_API_SECRET, access_token=TWITTER_ACCESS_TOKEN, access_token_secret=TWITTER_ACCESS_SECRET)


# Get your twitter account id

In [4]:

result = client.get_user(username=TWITTER_USERNAME)

In [12]:
twitter_id = result.data.id

# Get the users you are following

In [35]:
def get_users_account_is_following(twitter_id):
    following = []
    results = client.get_users_following(id=twitter_id, user_fields=USER_FIELDS, max_results=1000)
    following.extend(results.data)
    pagination_token = results.meta['next_token']
    
    while pagination_token:
        next_page_results = client.get_users_following(id=twitter_id, pagination_token=pagination_token, user_fields=USER_FIELDS, max_results=1000)
        following.extend(next_page_results.data)
        pagination_token=next_page_results.meta.get('next_token', "")
    
    return following

    

In [36]:

following_these_users = []

following = get_users_account_is_following(twitter_id)

print(len(following))


4292


# Get users following you

In [34]:
def get_users_following_account(twitter_id):
    followers = []
    results = client.get_users_followers(id=twitter_id, user_fields=USER_FIELDS, max_results=1000)
    followers.extend(results.data)
    pagination_token = results.meta['next_token']
    
    while pagination_token:
        next_page_results = client.get_users_followers(id=twitter_id, pagination_token=pagination_token, user_fields=USER_FIELDS, max_results=1000)
        followers.extend(next_page_results.data)
        pagination_token=next_page_results.meta.get('next_token', "")
    
    return followers


In [37]:
followers = get_users_following_account(twitter_id)
print(len(followers))

1978


# Create CSV Files

In [42]:
following_data = []

for user in following:
    setUserData = {
        'twitter_id': user.id,
        'name': user.name,
        'username': user.username,
        'bio': user.description,
        'url': user.url,
        'created_at': user.created_at,
        'number_accounts_followed': user.public_metrics['following_count'],
        'number_accounts_following': user.public_metrics['followers_count'],
        'number_of_tweets':user.public_metrics['tweet_count'],
        'profile_image_url': user.profile_image_url,
        'pinned_tweet_id': user.pinned_tweet_id,
        'location': user.location
    }
    following_data.append(setUserData)
    
following_dataframe = pd.DataFrame(following_data)

In [43]:
followers_data = []

for user in followers:
    setUserData = {
        'twitter_id': user.id,
        'name': user.name,
        'username': user.username,
        'bio': user.description,
        'url': user.url,
        'created_at': user.created_at,
        'number_accounts_followed': user.public_metrics['following_count'],
        'number_accounts_following': user.public_metrics['followers_count'],
        'number_of_tweets':user.public_metrics['tweet_count'],
        'profile_image_url': user.profile_image_url,
        'pinned_tweet_id': user.pinned_tweet_id,
        'location': user.location
    }
    followers_data.append(setUserData)
    
followers_dataframe = pd.DataFrame(followers_data)

In [45]:
following_dataframe.to_csv("following_users.csv")
followers_dataframe.to_csv("followed_by_users.csv")